# Libraries for this project

In [58]:
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd
import glob
from PIL import *
from natsort import natsorted, ns
from math import *
from os.path import join
from os import listdir
import shutil
from os import listdir
from os.path import isfile, join
from shutil import copytree
from shutil import move
import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.layers import Dense, Dropout, Activation,Flatten, Convolution2D,InputLayer, MaxPooling2D
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm
import skimage
from skimage import img_as_ubyte
from skimage import exposure


# Path Setting

In [59]:
# Path setting
dataset_path=r"D:\Divya_Project\59F_Dataset1"
video_path=r'D:\Divya_Project\59F/*.mp4'
phoneme_path=r'59F_phoneme'
word_path=r'59F_word'
path_video=r"D:\Divya_Project\59F"
dataset=r'Dataset0'
path_dataset_phoneme=r"D:\Divya_Project\Dataset_Phoneme"
path_dataset_word=r"D:\Divya_Project\Dataset_word"
png_num=59
path2=r'D:\Project_dataset\04M_Dataset1\04M_word/'
path1=r'D:\Project_dataset\Dataset_word\Dataset1/'
path4=r'D:\Project_dataset\04M_Dataset1\04M_phoneme/'
path3=r'D:\Project_dataset\Dataset_Phoneme\Dataset1/'

# Processing Videos

In [60]:
#Processing videos
if not os.path.exists(dataset_path):
    os.mkdir(dataset_path)
if not os.path.exists(path_dataset_phoneme):
    os.mkdir(path_dataset_phoneme)
if not os.path.exists(path_dataset_word):
    os.mkdir(path_dataset_word)
#DLIB pretrained model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#Initializing
i=0
count=0
end=True
frames=[]
videolist=glob.glob(video_path)

def process_video(videolist):
    #Processing all the videos in the folder
    for ii in videolist:
        cap = cv2.VideoCapture(ii)
        print("Files processing: {}-{}".format(ii,i))
        i+=1
        #Detecting face
        while cap is not None:
            ret,frame=cap.read()
            if frame is not None:
                gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
                faces = detector(gray)
                #Extracting the points in the mouth
                for k, d in enumerate(faces):
                    shape = predictor(frame, d)
                    mouthpoints_x = [shape.part(x).x for x in range(48,67)]
                    mouthpoints_y = [shape.part(x).y for x in range(48,67)]
                    max_x = max(mouthpoints_x)
                    min_x = min(mouthpoints_x)
                    max_y = max(mouthpoints_y)
                    min_y = min(mouthpoints_y)
                    #Cropping only the Lips with padded 25
                    padding = 25
                    crop_image = frame[min_y-padding:max_y+padding,min_x-padding:max_x+padding]
                    #Calculating the time of the frames
                    fps = cap.get(cv2.CAP_PROP_POS_MSEC)
                    #Saving the time of the frames for the next step
                    frames_sec=ceil(fps*10000)
                    a=("Sec."+str("{:06}".format(count))+"."+str(frames_sec))+"."+str(i-1)+"."+str(ii)
                    frames.append(a)
                    #Saving the images in the path
                    cv2.imwrite(dataset_path +"/"+str("{:02}".format(i))+"_"+str("{:06}".format(count))+".png",crop_image)
                    count+=1
            else:
                break
    cap.release()
    cv2.waitKey(0)
    cv2.destroyAllWindows()
process_video(videolist)

Files processing: D:\Divya_Project\59F\sa1.mp4-0
Files processing: D:\Divya_Project\59F\sa2.mp4-1
Files processing: D:\Divya_Project\59F\si511.mp4-2


# Preparing Ground truth label

# Preparing Ground truth label for phonemes


In [72]:
def prepare_groundtruth_label(dataset_path):
    #Preparing groud truth label for Words and Phonemes
    #Join the path and the frames time
    imagePaths = sorted([os.path.join(dataset_path, x) for x in os.listdir(dataset_path)])
    #Converting to dataframe
    images=pd.DataFrame(imagePaths)
    f=pd.DataFrame(frames)
    df1= pd.merge(f,images,left_index=True,right_index=True)
    df1.columns=["Sec","Img"]
    df1
    ##Extracting from 11th element
    df1['Sec'] = df1['Sec'].str[11:]
    ##Split by'.'
    new = df1["Sec"].str.split(".", n = 1, expand = True) 
    df1.drop(columns =["Sec"], inplace = True) 
    df1["Sec"]= new[0] 
    df1['List']=new[1]
    new = df1["List"].str.split(".", n = 1, expand = True)  
    df1['List']=new[1]
    new = df1["List"].str.split(".", n = 1, expand = True) 
    df1["File"]= new[0] 
    df1.drop(columns =["List"], inplace = True)
    ##Extracting from 17th element
    df1['File']=df1['File'].str[17:]
    return df1

In [73]:
def open_mlf_file():
    #Opening MLF file 
    with open(r'volunteer_labelfiles.mlf', 'r') as f:
        data = f.read().split('.')
    #Converting unstructured to structured data
    a=[s.replace('Q:/Videos/..','') for s in data]
    b=[s.replace('/Clips/straightcam/',' ') for s in a]
    d= [i.split('\n') for i in b]
    f = {e[1] : e[2:] for e in d}
    s = pd.Series(f)
    text=pd.DataFrame({'txt':s.index, 'list':s.values})
    return text

In [74]:
def clean_text_dataframe(text):
    #Grouping by txt
    df = text.groupby('txt').list.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
    df.columns = ['txt','list']
    #Splitting and cleaning txt
    new = df["txt"].str.split(" ", n = 1, expand = True) 
    df["M/F"]= new[0] 
    df['file']=new[1]
    new = df['list'].str.split(" ", n = 1, expand = True) 
    df["First_Sec"]= new[0] 
    df['Last_sec']=new[1]
    new = df['Last_sec'].str.split(" ", n = 1, expand = True) 
    df['Last_sec']=new[0]
    df['Phonemes']=new[1] 
    #Add \ inbetween file and give column name as M/F
    df['File']=df.apply(lambda x:'%s\%s' % (x['M/F'],x['file']),axis=1)
    df.drop(columns =["txt","list","M/F","file"], inplace = True) 
    df.dropna()
    #Drop the none value
    df = df.replace(to_replace='None', value=np.nan).dropna()
    return df

,First_Sec,Last_sec,Phonemes,File
0,0,9100000,sil,01M\sa1
1,9100000,10500000,sh,01M\sa1
2,10500000,11300000,iy,01M\sa1
3,11300000,11600000,hh,01M\sa1
4,11600000,12500000,ae,01M\sa1
5,12500000,12900000,d,01M\sa1
6,12900000,13500000,y,01M\sa1
7,13500000,14200000,uh,01M\sa1
8,14200000,14500000,r,01M\sa1
9,14500000,15300000,d,01M\sa1


In [76]:
def group_phonemes(df1):
    #Merge df and df1 with File as common
    df4=df.set_index('File').merge(df1, left_index=True, right_on='File')
    #SQL query (Extract only if Sec lies between  First and Last sec )
    df4 = df4.query('Sec >= First_Sec and Sec <= Last_sec')
    New=df4[["Phonemes","Img"]]
    N=New.set_index("Phonemes")
    New.Phonemes.unique()
    #Grouping based on Phonemes
    phonemes_image_path_dict=New.groupby("Phonemes")[["Img"]].apply(lambda x:x.values.tolist()).to_dict()
    return phonemes_image_path_dict, New

{'aa': [['D:\\Divya_Project\\59F_Dataset1\\01_000005.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000056.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000057.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000058.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000059.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000060.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000092.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000093.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000094.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000102.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000103.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000104.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000113.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000114.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000115.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000116.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\01_000117.png'],
  ['D:\\Divya_Project\\59F_Dataset1\\03_000304.png'],
  ['D:\\Divya_Project\

In [77]:
def copy_image_to_phonemes_or_words(dataset_path, phoneme_path,new_df, phonemes_image_path_dict):
    png_num=59
    src_path = dataset_path+'/'
    #Create a folder as imag if folder doesn't exist
    if not os.path.exists(os.path.join(src_path,phoneme_path)):
        os.mkdir(os.path.join(src_path,phoneme_path))
    dest_path = src_path+phoneme_path
    #Check for the unique phonemes in New
    ub = new_df.Phonemes.unique()    
    #For each unique phonemes create a folder if it doesn't exist
    for u in ub:
        if not os.path.exists(os.path.join(dest_path,u)):
            os.mkdir(os.path.join(dest_path,u))
    count=0
    #Copy the image to the corresponding phonemes
    for k,v in phonemes_image_path_dict.items():
        filename=k
        for i in v:
            for ii in i:
                #filename=k
                iii=cv2.imread(ii)
                count+=1
                cv2.imwrite(dest_path+"/"+filename+"/"+str("{:06}".format(count))+'-'+str("{:02}".format(png_num))+".png",iii)

In [78]:
def prepare_dataset_phonemes_words(path_dataset_phoneme, dataset, new_df):
    #Copying the images in a seperate folder to create folds for cv
    src_path = path_dataset_phoneme+'/'
    if not os.path.exists(os.path.join(src_path,dataset)):
        os.mkdir(os.path.join(src_path,dataset))
    dest_path = src_path+dataset
    ub = new_df.Phonemes.unique()    
    for u in ub:
        if not os.path.exists(os.path.join(dest_path,u)):
            os.mkdir(os.path.join(dest_path,u))
    #Copy the image to the corresponding phonemes
    count=0
    for k,v in df6.items():
        filename=k
        for i in v:
            for ii in i:
                #filename=k
                iii=cv2.imread(ii)
                count+=1
                cv2.imwrite(dest_path+"/"+filename+"/"+str("{:06}".format(count))+'-'+str("{:02}".format(png_num))+".png",iii)

In [ ]:
def prepare_gt_label_phonemes(dataset, dataset_path, phoneme_path, path_dataset_phoneme):
    df1 = prepare_groundtruth_label(dataset_path)
    text = open_mlf_file()
    df = clean_text_dataframe(text)
    phonemes_image_path_dict, new_df = group_phonemes(df1)
    copy_image_to_phonemes_or_words(dataset_path, phoneme_path,new_df, phonemes_image_path_dict)
    prepare_dataset_phonemes_words(path_dataset_phoneme, dataset, new_df)
prepare_gt_label_phonemes(dataset, dataset_path, phoneme_path, path_dataset_phoneme)

# Preparing grouth truth label for Words

In [79]:
def create_df_for_words(imagePaths, frames, path_video):
    images=pd.DataFrame(imagePaths)
    #Converting into Dataframes
    f=pd.DataFrame(frames)
    df1= pd.merge(f,images,left_index=True,right_index=True)
    df1.columns=["Sec","Img"]
    ##Extracting from 11th element
    df1['Sec'] = df1['Sec'].str[11:]
    new = df1["Sec"].str.split(".", n = 1, expand = True) 
    df1["Sec"]= new[0] 
    df1['Num']=new[1]
    new = df1["Num"].str.split(".", n = 1, expand = True) 
    df1["Num"]= new[0] 
    df1["Sec"] = pd.to_numeric(df1["Sec"])
    df1["Num"] = pd.to_numeric(df1["Num"])
    df1['Sec']=df1['Sec'].apply(lambda x: x/1000)
    df1['Sec'] = df1['Sec'].apply(np.ceil)
    #Join the path with .txt file
    files_txt = sorted([os.path.join(path_video, x) for x in os.listdir(path_video) if x[-3:] =='txt'])
    df3 = pd.DataFrame()
    #Give a variable 0,1,2.. to each new .txt file
    for x,y in enumerate(files_txt):  
        data =pd.read_csv(y,sep=" ", header = None)
        data['new'] = x
        df3=pd.concat([df3,data])
    df3.columns = ['First_sec', 'Last_sec','Word',"New"]
    return df3, df1


In [80]:
def group_based_on_words(df, df1):
    #Merge df and df4 with Num as common
    df4=df.set_index('New').merge(df1, left_index=True, right_on='Num')
    #SQL query (Extract only if Sec lies between  First and Last sec )
    df4 = df4.query('Sec >= First_sec and Sec <= Last_sec')
    New=df4[["Word","Img"]]
    N=New.set_index("Word")
    New.Word.unique()
    #Grouping based on Word
    df6=New.groupby("Word")[["Img"]].apply(lambda x:x.values.tolist()).to_dict()
    return df6, New

In [ ]:
def prepare_gt_label_phonemes(dataset, imagePaths, frames, path_video, dataset_path, word_path, path_dataset_word):
    df, df1 = create_df_for_words(imagePaths, frames, path_video)
    words_image_path_dict, new_df = group_based_on_words(df, df1)
    copy_image_to_phonemes(dataset_path, word_path,new_df, words_image_path_dict)
    prepare_dataset_phonemes(path_dataset_word, dataset, new_df)
prepare_gt_label_phonemes(dataset, imagePaths, frames, path_video, dataset_path, word_path, path_dataset_word)

# Joining two folds for cros validation

In [ ]:
#For Cross-Validation, joining two folds

unique_path1 = [f for f in os.listdir(path1) if not f.startswith('.')]
unique_path2 = [f for f in os.listdir(path2) if not f.startswith('.')]
unique_path3 = [f for f in os.listdir(path3) if not f.startswith('.')]
unique_path4 = [f for f in os.listdir(path4) if not f.startswith('.')]


In [ ]:
#If there is any unmatch in two folds , copy the folder 
match=[x for x in unique_path2 if x not in unique_path1]
if match != []:
    for j in match:
        shutil.copytree(os.path.join(path2,j),os.path.join(path1,j))
match = [x for x in os.listdir(path1) if x in os.listdir(path2)]
# iterate through each common folder name
for i in match:
    root = os.path.join(path2,i)
    dest = os.path.join(path1,i)
## copy the imgs from the common folder name of path to b
    for im in os.listdir(root):
        shutil.copy(os.path.join(root,im), os.path.join(dest,im))
    
match1=[x for x in unique_path4 if x not in unique_path3]
if match1 != []:
    for j in match1:
        shutil.copytree(os.path.join(path4,j),os.path.join(path3,j))
match1 = [x for x in os.listdir(path3) if x in os.listdir(path4)]
# iterate through each common folder name
for i in match1:
    root = os.path.join(path4,i)
    dest = os.path.join(path3,i)
## copy the imgs from the common folder name of path to b
    for im in os.listdir(root):
        shutil.copy(os.path.join(root,im), os.path.join(dest,im))   

# Creating OOV folder if there is any unmatch in training and testing dataset

In [ ]:
#Checking for same labels name in both training and testing, if not creating OOV folder
test_path = r'D:\Project_dataset\testing/imag/'
train_path= r'D:\Project_dataset\training/imag/'
folders_train = [f for f in os.listdir(train_path) if not f.startswith('.')]
folders_test = [f for f in os.listdir(test_path) if not f.startswith('.')]
#TRAINING
if not os.path.exists(os.path.join(train_path,'OOV')):
    os.mkdir(os.path.join(train_path,'OOV'))
path = r'D:\Project_dataset\training\imag\OOV'
#If there is any unmatch, create OOV folder in the training set
match=[x for x in folders_train if x not in folders_test]
for j in range(len(match)):
    shutil.move(os.path.join(train_path,match[j]),path)
    
for r, d, f in os.walk(path):
    for a in d:
        for filename in listdir(join(path, a)):
            move(join(path, a, filename), join(path, filename))
for file_name in listdir(path):
        if not file_name.endswith('.png'):
            shutil.rmtree(os.path.join(path,file_name))
#Testing
if not os.path.exists(os.path.join(test_path,'OOV')):
    os.mkdir(os.path.join(test_path,'OOV'))
path = r'D:\Project_dataset\training\imag\OOV'
#If there is any unmatch, create OOV folder in the testing set
match=[x for x in folders_test if x not in folders_train]
for j in range(len(match)):
    shutil.move(os.path.join(test_path,match[j]),path)
for r, d, f in os.walk(path):
    for a in d:
        for filename in listdir(join(path, a)):
            move(join(path, a, filename), join(path, filename))
for file_name in listdir(path):
        if not file_name.endswith('.png'):
            shutil.rmtree(os.path.join(path,file_name)) 

# Preparing images to feed into the model

In [ ]:
test_path = r'E:\Fold_phoneme2/'
train_path = r'E:\Fold13_phoneme/'
img_cols,img_rows=100,100

In [ ]:
#Converting images to int64
#Training
folders_train = [f for f in os.listdir(train_path) if not f.startswith('.')]
y_train = np.empty((0))
for j in range(len(folders_train)):
        path_to_frames = os.path.join(train_path, folders_train[j])
        files = os.listdir(path_to_frames)
        natsorted(files, alg=ns.IGNORECASE)
        labels = np.repeat(j, len(files))
        y_train = np.append(y_train, labels, axis=0)
y_train=y_train.astype(int)

X_train=[]
   
for folderName in os.listdir(train_path):
    for image_filename in tqdm(os.listdir(train_path + folderName)):
        img_file = cv2.imread(train_path + folderName + '/' + image_filename,cv2.IMREAD_GRAYSCALE)
        if img_file is not None:
            img_file = skimage.transform.resize(img_file, (img_rows, img_cols ,1))            
            img = img_as_ubyte(img_file)
            img_arr = np.asarray(img) #with equalization
            X_train.append(img_arr)
X_train = np.asarray(X_train)

class_train=len(folders_train)
y_train = keras.utils.to_categorical(y_train,class_train)
X_train = X_train.reshape(X_train.shape[0],img_rows, img_cols ,1)


print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print('y_train shape:', y_train.shape)



In [ ]:
#TESTING

folders_t = [f for f in os.listdir(test_path) if not f.startswith('.')]
y_test = np.empty((0))
for j in range(len(folders_t)):
        path_to_frames = os.path.join(test_path, folders_t[j])
        files = os.listdir(path_to_frames)
        natsorted(files, alg=ns.IGNORECASE)
        labels = np.repeat(j, len(files))
        y_test = np.append(y_test, labels, axis=0)
y_test=y_test.astype(int)

X_test=[]
   
for folderName in os.listdir(test_path):
    for image_filename in tqdm(os.listdir(test_path + folderName)):
        img_file = cv2.imread(test_path + folderName + '/' + image_filename,cv2.IMREAD_GRAYSCALE)
        if img_file is not None:
            img_file = skimage.transform.resize(img_file, (img_rows, img_cols ,1))            
            img = img_as_ubyte(img_file)
            img_arr = np.asarray(img)
            X_test.append(img)
X_test = np.asarray(X_test)
y_test = keras.utils.to_categorical(y_test,class_train)
X_test = X_test.reshape(X_test.shape[0],img_rows, img_cols ,1)
print('x_test shape:', X_test.shape)
print(X_test.shape[0], 'test samples')
print('y_test shape:', y_test.shape)



# Creating CNN: Model 1

In [ ]:
#Model with 4 layers with rmsprop optimizer
img_input = layers.Input(shape=((img_rows, img_cols, 1)))
input_shape=(img_rows, img_cols,1)
model = Sequential()
model.add(InputLayer(input_shape=((img_rows, img_cols,1))))

model.add(Convolution2D(filters=32, 
                        kernel_size=(3, 3), 
                        activation='relu', 
                        input_shape=input_shape,
                        padding='same'
                        ))
model.add(MaxPooling2D(pool_size=(2, 2)
          , padding='same'))
    
model.add(Convolution2D(filters=64, 
                        kernel_size=(1, 1), 
                        activation='relu',
                        #input_shape=input_shape,
                        padding='same'
                        ))
model.add(MaxPooling2D(pool_size=(2, 2)
          , padding='same'))

model.add(Convolution2D(filters=128, 
                        kernel_size=(1, 1), 
                        activation='relu',
                        #input_shape=input_shape,
                         padding='same'
                        ))

model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))


model.add(Dense(64, activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Convolution2D(filters=256, 
                        kernel_size=(1, 1), 
                        activation='relu', 
                        #input_shape=input_shape,
                        padding='same'
                        ))

model.add(Flatten())
#model.add(Dropout(0.25))
#model.add(Dense(1024, activation='softmax'))
model.add(Dropout(0.5))

#model.add(Dense(num_classes, activation='softmax'))
#model.add(Flatten())

model.add(Dense(class_train, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy','mse'])

score=model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

print('Train loss:', score)

history=model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test))

predicts=model.predict(X_test,
              batch_size=1000,
              verbose=1)
y_test_non_category = [ np.argmax(t) for t in y_test ]
y_predict = [ np.argmax(t) for t in predicts ]
    
# Evaluate on the test data

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
#Confusion matrix
print('Confusion Matrix')
cm = confusion_matrix(y_test_non_category, y_predict)
print(cm)
#Classification report
print('Classification Report')
print(classification_report(y_test_non_category,y_predict))


In [ ]:
print(history.history.keys())
# Plotting accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

# Plotting loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# Data augmentation 

In [ ]:
#Working
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range = 40,
    fill_mode='nearest',
    width_shift_range = 0.2,                  
    height_shift_range = 0.2,                  
    rescale = 1./255,                  
    shear_range = 0.2,                  
    zoom_range = 0.2,                     
    horizontal_flip = True)
IMG_SIZE=100
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE,IMG_SIZE),
    color_mode="grayscale",interpolation='nearest',
    class_mode='categorical',
    batch_size = 100)
validation_generator = validation_datagen.flow_from_directory(
    test_path,
    color_mode="grayscale",interpolation='nearest',
    target_size=(IMG_SIZE,IMG_SIZE),
    class_mode='categorical',
    batch_size = 100)

In [ ]:
 
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=100)
#model.save_weights('cnn_baseline.h5')
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Creating CNN model 2

In [ ]:
#CNN model with 6 layers and adam optimiser

# Our input feature map is 100x100x1: 100 X 100 for the image pixels, and 1 for grayscale channel
input_shape=(img_rows, img_cols,1)
model = Sequential()

model.add(InputLayer(input_shape=((img_rows, img_cols,1))))


model.add(Convolution2D(filters=32, 
                        kernel_size=(3, 3), 
                        activation='relu', padding='same',
                        input_shape=input_shape,
                        name='conv1'
                        ))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
model.add(Convolution2D(filters=32, 
                        kernel_size=(3, 3), 
                        activation='relu', padding='same',
                        input_shape=input_shape
                        ))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))


model.add(Convolution2D(filters=64, 
                        kernel_size=(3, 3), 
                        activation='relu', 
                        input_shape=input_shape
                        ))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Convolution2D(filters=64, 
                        kernel_size=(3, 3), 
                        activation='relu', padding='same',
                        input_shape=input_shape
                        ))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Convolution2D(filters=128, 
                        kernel_size=(3, 3), 
                        activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    
model.add(Convolution2D(filters=128, 
                        kernel_size=(3, 3), padding='same',
                        activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))


model.add(Flatten())

#Adding dropout to prevent overfitting
#model.add(Dropout(0.25))
model.add(Dropout(0.5))

model.add(Dense(class_train, activation='softmax'))

model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy','mse'])

import matplotlib.pyplot as plt
batch_size = 1000
epochs=10
   
history=model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test))

predicts=model.predict(X_test,
              batch_size=1000,
              verbose=1)
y_test_non_category = [ np.argmax(t) for t in y_test ]
y_predict = [ np.argmax(t) for t in predicts ]

# Evaluate on test data

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.figure(figsize=(10,15),dpi=50)  
   
 #Plotting accuracy  
   
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
   
 # Plotting loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()  